### classify stackoverflow posts by considering each document a simple mean of its individual word embeddings (glove trained on stackoverflow corpus), then using individual tag probabilities to assign tags to samples

In [1]:
import csv
import os
import re
import sys
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from nltk.tokenize import StanfordTokenizer

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

Using TensorFlow backend.


In [2]:
SEED=np.random.randint(1,1000)
SEED

113

In [3]:
np.random.seed(SEED)

In [4]:
texts, labels = files_helper.read_stackoverflow_sample_small_stanford_tokenized()

In [5]:
MAX_NB_WORDS = 200000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
EMBEDDING_DIM=100
NUM_EPOCHS=10
TOKENIZER_FILTERS='' # I will perform tokenization myself

In [6]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters=TOKENIZER_FILTERS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [7]:
[inverse_word_index[idx] for idx in sequences[1]]

['replacing',
 'asp',
 'datagrid',
 'control',
 'to',
 'custom',
 'jquery',
 'control',
 '(',
 'telerik',
 ',',
 'or',
 'other',
 'library',
 ')',
 '.',
 'we',
 'have',
 'a',
 'traditional',
 'webform',
 'aspx',
 'application',
 '.',
 'we',
 'are',
 'using',
 'so',
 'many',
 'asp',
 'server',
 'controls',
 'in',
 'our',
 'application',
 'which',
 'lacks',
 'in',
 'ui',
 'look',
 'n',
 'feel',
 '.',
 'i',
 'have',
 'been',
 'looking',
 'into',
 'telerik',
 'and',
 'other',
 'asp',
 'custom',
 'control',
 'library',
 '.',
 'we',
 'are',
 'also',
 'looking',
 'for',
 'open',
 'source',
 'library',
 'which',
 'doesnt',
 'come',
 'with',
 'license',
 '.',
 'i',
 'want',
 'to',
 'ask',
 'how',
 'can',
 'we',
 'start',
 'replacing',
 'all',
 'the',
 'asp',
 '.',
 'net',
 'control',
 'to',
 'any',
 'good',
 'ui',
 'custom',
 'control',
 'library',
 '.',
 'as',
 'we',
 'want',
 'to',
 'change',
 'as',
 'much',
 'less',
 'our',
 'code',
 '.',
 'i',
 'need',
 'pointers',
 'to',
 'open',
 'source'

In [8]:
truncated_labels = tags_helper.truncate_labels(labels,LABELS_MIN_DOC_COUNT)

In [9]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(truncated_labels)

In [10]:
# tag position => fraction of docs having that tag
# snooping
tag_incidence_index = tags_helper.get_incidence_index(binary_labels)

In [11]:
# word => embedding
embeddings_index = files_helper.read_glove_stackoverflow(EMBEDDING_DIM)

In [12]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [13]:
embedding_matrix.shape

(221514, 100)

In [14]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)


indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_1 = binary_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
Y_train = labels_1[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_1[-num_validation_samples:]

data.shape,labels_1.shape

((32000, 1000), (32000, 1209))

In [46]:
%%time

num_labels = labels_1.shape[1]

model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           trainable = True))

model.add(GlobalAvgPool1D())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam')

# loss doesn't get better after 5 epochs
model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    verbose=1,
                    validation_data=(X_val,Y_val))

Train on 25600 samples, validate on 6400 samples
Epoch 1/10
25600/25600 [==============================] - 49s - loss: 0.0410 - val_loss: 0.0118
Epoch 2/10
25600/25600 [==============================] - 49s - loss: 0.0128 - val_loss: 0.0117
Epoch 3/10
25600/25600 [==============================] - 49s - loss: 0.0125 - val_loss: 0.0115
Epoch 4/10
25600/25600 [==============================] - 49s - loss: 0.0124 - val_loss: 0.0115
Epoch 5/10
25600/25600 [==============================] - 49s - loss: 0.0123 - val_loss: 0.0115
Epoch 6/10
25600/25600 [==============================] - 49s - loss: 0.0121 - val_loss: 0.0114
Epoch 7/10
25600/25600 [==============================] - 49s - loss: 0.0120 - val_loss: 0.0115
Epoch 8/10
25600/25600 [==============================] - 48s - loss: 0.0119 - val_loss: 0.0113
Epoch 9/10
25600/25600 [==============================] - 48s - loss: 0.0118 - val_loss: 0.0113
Epoch 10/10
25600/25600 [==============================] - 48s - loss: 0.0116 - val_los

In [47]:
Y_train_pred = []

for x_train in X_train:
    y_train_pred = model.predict(x_train.reshape(1,-1))
    Y_train_pred.append(y_train_pred)

Y_train_pred = np.vstack(Y_train_pred)

In [48]:
tag_probability_index = tags_helper.get_probability_index(Y_train_pred)

In [49]:
def get_scores(predicted_probabilities, predicted_indices,debug=False):
    """
    predicted_probabilities (output of n sigmoid output units)
    predicted_indices (according to some strategy, e.g. static threshold, etc)
    """
    
    # indices that are turned on (equal 1)
    active_indices = predicted_indices.ravel().nonzero()[0]
    
    if debug:
        print("active_indices: {0}".format(active_indices))
    
    scores_for_active_indices = predicted_probabilities[active_indices]
    
    scores_for_active_indices_as_matrix = scores_for_active_indices.reshape(1,-1)
    
    return scores_for_active_indices_as_matrix
    

In [50]:
lb.transform([set(['c++'])]).ravel().nonzero()[0]

array([142])

In [51]:
i=555

tag_incidence_index[535] # java
tag_incidence_index[1059] # twitter-bootstrap
tag_incidence_index[692] # mysql
tag_incidence_index[i],tag_probability_index[i] # c++

(0.00050000000000000001, 0.00022992265)

In [72]:
# sample result for a couple of test cases
num_test_cases = 10
STRATEGY= 'absolute_difference_wrt_estimated_tag_probability'
THRESHOLD=0.0
PROB_INDEX = tag_probability_index
INCIDENCE_INDEX = tag_incidence_index
LIMIT=5

for i in np.random.randint(low=0, high=len(Y_val), size=num_test_cases):

    actual_label_indices = Y_val[i].reshape(1,-1)
    
#     print(actual_label_indices.ravel().nonzero()[0])
    
    actual_labels = lb.inverse_transform(actual_label_indices)
    actual_labels_tpl = actual_labels[0]
    
    predicted_tag_probabilities = model.predict(X_val[i].reshape(1,-1)).ravel()
              
    predicted_label_indices = tags_helper.get_tag_assignment(
        STRATEGY,
        predicted_tag_probabilities,
        probability_threshold=THRESHOLD,
        tag_probability_index=PROB_INDEX,
        tag_incidence_index=INCIDENCE_INDEX,
        limit=LIMIT)
         
    predicted_label_scores_mat = get_scores(
        predicted_tag_probabilities,
        predicted_label_indices)
    
    actual_label_scores_mat = get_scores(
        predicted_tag_probabilities,
        actual_label_indices)
    
    predicted_label_scores_arr = predicted_label_scores_mat.ravel()
    predicted_labels = lb.inverse_transform(predicted_label_indices)
    predicted_labels_tpl = predicted_labels[0]
      
    predicted_tags_and_scores = sorted(list(zip(predicted_labels_tpl,predicted_label_scores_arr)), key=lambda tpl: tpl[1], reverse=True)
    
    actual_label_scores_arr = actual_label_scores_mat.ravel()
    actual_tags_and_scores = list(zip(actual_labels_tpl,actual_label_scores_arr))

#     print(actual_labels)    
    print(actual_labels_tpl)
    print(actual_tags_and_scores)
    print(predicted_tags_and_scores)
    print('\n')


('algorithm', 'c')
[('algorithm', 0.004225546), ('c', 0.017661804)]
[('java', 0.12417604), ('c#', 0.11598413), ('android', 0.11426815), ('c++', 0.043450218), ('.net', 0.021286447)]


('android',)
[('android', 0.32248688)]
[('android', 0.32248688), ('java', 0.30222484), ('c#', 0.19752023), ('hibernate', 0.0265132), ('listview', 0.022677874)]


('curl', 'ios', 'swift')
[('curl', 0.0028719448), ('ios', 0.033229381), ('swift', 0.0085833101)]
[('javascript', 0.08305303), ('python', 0.060669299), ('php', 0.060174718), ('html', 0.038927305), ('css', 0.030812638)]


('google-chrome', 'windows')
[('google-chrome', 0.0040159943), ('windows', 0.0091166813)]
[('c#', 0.10156003), ('c++', 0.041578036), ('ios', 0.035148226), ('objective-c', 0.023078233), ('.net', 0.022047898)]


('pdf',)
[('pdf', 0.0015182955)]
[('javascript', 0.081311353), ('python', 0.05678989), ('html', 0.036641974), ('sql', 0.029634891), ('css', 0.028391367)]


('angularjs', 'mysql', 'php')
[('angularjs', 0.032556839), ('mysql', 

In [73]:
X_test_list = []

for validation_features in X_val:
    X_test_list.append(validation_features.reshape(1,-1))

X_test = np.vstack(X_test_list)

In [74]:
Y_pred_lst = [model.predict(x_test.reshape(1,-1)) for x_test in X_test]

In [75]:
Y_pred_calculated_lst = [tags_helper.get_tag_assignment(
    STRATEGY,
    y_pred,
    probability_threshold=THRESHOLD,
    tag_probability_index=PROB_INDEX,
    tag_incidence_index=INCIDENCE_INDEX,
    limit=LIMIT) for y_pred in np.vstack(Y_pred_lst)]

Y_pred = np.vstack(Y_pred_calculated_lst)

Y_val.shape,Y_pred.shape

((6400, 1209), (6400, 1209))

In [76]:
micro_f1 = metrics_helper.calculate_multilabel_metrics(Y_val,Y_pred)
print('micro-averaged F1 score (validation set) is {0}'.format(micro_f1))

micro_prec = metrics_helper.calculate_multilabel_metrics(Y_val, Y_pred, metric='precision')
print('micro-averaged precision score (validation set) is {0}'.format(micro_prec))

micro_rec = metrics_helper.calculate_multilabel_metrics(Y_val, Y_pred, metric='recall')
print('micro-averaged recall score (validation set) is {0}'.format(micro_rec))

micro-averaged F1 score (validation set) is 0.14337669005647785
micro-averaged precision score (validation set) is 0.10471875
micro-averaged recall score (validation set) is 0.22727889310906133
